In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

category_columns = [
    "Unlawful detention",
    "Human trafficking",
    "Enslavement",
    "Willful killing of civilians",
    "Mass execution",
    "Kidnapping",
    "Extrajudicial killing",
    "Forced disappearance",
    "Damage or destruction of civilian critical infrastructure",
    "Damage or destruction, looting, or theft of cultural heritage",
    "Military operations (battle, shelling)",
    "Gender-based or other conflict-related sexual violence",
    "Violent crackdowns on protesters/opponents/civil rights abuse",
    "Indiscriminate use of weapons",
    "Torture or indications of torture",
    "Persecution based on political, racial, ethnic, gender, or sexual orientation",
    "Movement of military, paramilitary, or other troops and equipment"
]

# 2) Custom Dataset class for articles
class ArticleDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_df = pd.read_csv("train.csv")  
val_df   = pd.read_csv("val.csv")    
test_df  = pd.read_csv("test.csv")  

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the text columns
train_encodings = tokenizer(
    list(train_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)
val_encodings   = tokenizer(
    list(val_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)
test_encodings  = tokenizer(
    list(test_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)

# Extract labels (multi-label targets in your category columns)
train_labels = train_df[category_columns].values
val_labels   = val_df[category_columns].values
test_labels  = test_df[category_columns].values

# Create Dataset objects
train_dataset = ArticleDataset(train_encodings, train_labels)
val_dataset   = ArticleDataset(val_encodings, val_labels)
test_dataset  = ArticleDataset(test_encodings, test_labels)


In [ ]:
# Note: num_labels = number of category columns for multi-label classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=len(category_columns)
)

# Define compute_metrics for multi-label classification
def compute_metrics(p):
    # p.predictions are logits; p.label_ids are the ground truth
    preds = torch.sigmoid(torch.tensor(p.predictions))  # Convert logits to probabilities
    preds = (preds > 0.5).int().cpu().numpy() 
    labels = torch.tensor(p.label_ids).cpu().numpy()
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels,
        preds,
        average='weighted'
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    eval_strategy="epoch",           # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save model at the end of each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",      # Use F1 score for best model
    logging_dir='./logs'
)

# Initialize Trainer with training and validation sets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("./bert-multiclass-model")


In [ ]:
def evaluate_model(dataset, threshold=0.5):
    loader = DataLoader(dataset, batch_size=16)
    model.eval()  # Set model to eval mode
    
    preds_list = []
    labels_list = []
    
    with torch.no_grad():
        for batch in loader:
            inputs = {
                key: val.to(model.device) for key, val in batch.items() if key != 'labels'
            }
            outputs = model(**inputs)
            logits = outputs.logits
            # Convert logits to probabilities
            probs = torch.sigmoid(logits).cpu().numpy()
            # Apply threshold
            preds = (probs > threshold).astype(int)
            
            preds_list.extend(preds)
            labels_list.extend(batch['labels'].cpu().numpy())
    
    preds_array = np.array(preds_list)
    labels_array = np.array(labels_list)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels_array, preds_array, average='weighted')
    acc = accuracy_score(labels_array, preds_array)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Run evaluation on the test dataset
test_results = evaluate_model(test_dataset)
print("Final Test Set Evaluation Results:", test_results)
